In [2]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Employment Rights and Legal Advice Chatbot Development\n",
    "\n",
    "This notebook demonstrates the development and testing of our Employment Rights and Legal Advice Chatbot using the Gen AI Llama-3.1-70b-versatile model on GroqCloud."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Install required libraries\n",
    "!pip install groq streamlit transformers torch SpeechRecognition pyttsx3"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "import os\n",
    "import json\n",
    "from groq import Groq\n",
    "from transformers import AutoTokenizer, AutoModel\n",
    "import speech_recognition as sr\n",
    "import pyttsx3"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Load configuration\n",
    "with open('config.json', 'r') as config_file:\n",
    "    config = json.load(config_file)\n",
    "\n",
    "# Set up Groq API key\n",
    "os.environ[\"GROQ_API_KEY\"] = \"your_api_key_here\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "class Chatbot:\n",
    "    def __init__(self):\n",
    "        self.client = Groq(api_key=os.environ[\"GROQ_API_KEY\"])\n",
    "\n",
    "    def generate_response(self, legal_analysis):\n",
    "        prompt = f\"\"\"\n",
    "        You are an AI assistant specializing in employment rights and labor laws. \n",
    "        Based on the following legal analysis, provide a clear and concise response to the user's query:\n",
    "\n",
    "        {legal_analysis}\n",
    "\n",
    "        Remember to:\n",
    "        1. Be informative but not overly technical\n",
    "        2. Provide actionable advice when appropriate\n",
    "        3. Encourage the user to seek professional legal counsel for complex issues\n",
    "\n",
    "        Response:\n",
    "        \"\"\"\n",
    "\n",
    "        chat_completion = self.client.chat.completions.create(\n",
    "            messages=[\n",
    "                {\n",
    "                    \"role\": \"user\",\n",
    "                    \"content\": prompt,\n",
    "                }\n",
    "            ],\n",
    "            model=config['model'],\n",
    "            max_tokens=config['max_response_length'],\n",
    "            temperature=config['temperature']\n",
    "        )\n",
    "\n",
    "        return chat_completion.choices[0].message.content\n",
    "\n",
    "chatbot = Chatbot()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "class LegalAnalyzer:\n",
    "    def __init__(self):\n",
    "        with open('data/labor_laws.json', 'r') as f:\n",
    "            self.labor_laws = json.load(f)\n",
    "        with open('data/case_studies.json', 'r') as f:\n",
    "            self.case_studies = json.load(f)\n",
    "\n",
    "    def analyze(self, query):\n",
    "        relevant_laws = self._find_relevant_laws(query)\n",
    "        relevant_cases = self._find_relevant_cases(query)\n",
    "        \n",
    "        analysis = f\"Relevant Laws:\\n{relevant_laws}\\n\\nRelevant Case Studies:\\n{relevant_cases}\"\n",
    "        return analysis\n",
    "\n",
    "    def _find_relevant_laws(self, query):\n",
    "        # Implement logic to find relevant laws based on the query\n",
    "        # This is a placeholder implementation\n",
    "        return \"Placeholder: Relevant labor laws would be listed here.\"\n",
    "\n",
    "    def _find_relevant_cases(self, query):\n",
    "        # Implement logic to find relevant case studies based on the query\n",
    "        # This is a placeholder implementation\n",
    "        return \"Placeholder: Relevant case studies would be listed here.\"\n",
    "\n",
    "legal_analyzer = LegalAnalyzer()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "class LanguageProcessor:\n",
    "    def __init__(self):\n",
    "        self.tokenizer = None\n",
    "        self.model = None\n",
    "        self.language = \"en\"\n",
    "\n",
    "    def set_language(self, language):\n",
    "        language_code = {\"English\": \"en\", \"Español\": \"es\", \"Français\": \"fr\"}\n",
    "        self.language = language_code[language]\n",
    "        self._load_model()\n",
    "\n",
    "    def _load_model(self):\n",
    "        model_path = f\"data/language_models/{self.language}_model.bin\"\n",
    "        self.tokenizer = AutoTokenizer.from_pretrained(model_path)\n",
    "        self.model = AutoModel.from_pretrained(model_path)\n",
    "\n",
    "    def process(self, text):\n",
    "        # Tokenize and process the input text\n",
    "        tokens = self.tokenizer(text, return_tensors=\"pt\")\n",
    "        outputs = self.model(**tokens)\n",
    "        # Further processing logic here\n",
    "        return outputs\n",
    "\n",
    "language_processor = LanguageProcessor()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "class VoiceInterface:\n",
    "    def __init__(self):\n",
    "        self.recognizer = sr.Recognizer()\n",
    "        self.engine = pyttsx3.init()\n",
    "\n",
    "    def listen(self):\n",
    "        with sr.Microphone() as source:\n",
    "            print(\"Listening...\")\n",
    "            audio = self.recognizer.listen(source)\n",
    "            try:\n",
    "                text = self.recognizer.recognize_google(audio)\n",
    "                return text\n",
    "            except sr.UnknownValueError:\n",
    "                return \"Sorry, I couldn't understand that.\"\n",
    "            except sr.RequestError:\n",
    "                return \"Sorry, there was an error processing your request.\"\n",
    "\n",
    "    def speak(self, text):\n",
    "        self.engine.say(text)\n",
    "        self.engine.runAndWait()\n",
    "\n",
    "voice_interface = VoiceInterface()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "def process_query(query, language=\"English\"):\n",
    "    language_processor.set_language(language)\n",
    "    processed_input = language_processor.process(query)\n",
    "    legal_analysis = legal_analyzer.analyze(processed_input)\n",
    "    response = chatbot.generate_response(legal_analysis)\n",
    "    return response\n",
    "\n",
    "# Test the chatbot\n",
    "test_query = \"What are my rights regarding overtime pay?\"\n",
    "response = process_query(test_query)\n",
    "print(f\"Query: {test_query}\")\n",
    "print(f\"Response: {response}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Test voice interface\n",
    "print(\"Please speak your question about employment rights:\")\n",
    "voice_query = voice_interface.listen()\n",
    "print(f\"You said: {voice_query}\")\n",
    "\n",
    "if voice_query != \"Sorry, I couldn't understand that.\" and voice_query != \"Sorry, there was an error processing your request.\":\n",
    "    response = process_query(voice_query)\n",
    "    print(f\"Chatbot response: {response}\")\n",
    "    voice_interface.speak(response)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Employment Rights and Legal Advice Chatbot Development\n',
    '\n',
    'This notebook demonstrates the development and testing of our Employment Rights and Legal Advice Chatbot using the Gen AI Llama-3.1-70b-versatile model on GroqCloud.']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'source': ['# Install required libraries\n',
    '!pip install groq streamlit transformers torch SpeechRecognition pyttsx3']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'source': ['import os\n',
    'import json\n',
    'from groq import Groq\n',
    'from transformers import AutoTokenizer, AutoModel\n',
    'import speech_recognition as sr\n',
    'import pyttsx3']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'source': ['# Load configuration\n',
    "with open('config.json', 'r') as config_file:\n",
    '    config = json.load(config_file)